# RSI_Hedge_v2.7

## Setup

### Packages

In [1]:
from utils.packages import *

### Inputs and Parameters

#### Read Yaml files

In [2]:
temp_file = 'config/access_keys.yaml'
with open(temp_file) as temp_file:
    config = yaml.load(temp_file)
    
temp_file = 'config/currencies.yaml'
with open(temp_file) as temp_file:
    currs = yaml.load(temp_file)        

## Functions

In [3]:
#--------------------------------------------------------------------------------------------------------------------------
# Get timestamp of the price and segregate it
def get_date_time(resp):
    global data
    time_stamp = resp['time']
    data['ts_date_val'], full_time = time_stamp.split(sep = 'T')
    data['ts_time_val'], _ = full_time.split(sep = '.')

    data['ts_date_val'] = datetime.strptime(data['ts_date_val'], '%Y-%m-%d')
    data['ts_time_val'] = datetime.strptime(data['ts_time_val'], '%H:%M:%S')

    tot_ts = datetime.combine(datetime.date(data['ts_date_val']), datetime.time(data['ts_time_val']))
    tot_ts = tot_ts + timedelta(hours=8)
    data['tot_ts'] = tot_ts.strftime("%Y-%b-%d, %I:%M:%S (%p)")
      
    t2 = datetime.now()    
    data['time_diff'] = (t2 - tot_ts).total_seconds()
    
    return()
#==========================================================================================================================


#--------------------------------------------------------------------------------------------------------------------------
# Get bid and ask prices
def get_prices(resp):    
    global data
    data['price_bid'] = float(resp['bids'][0]['price'])    
    data['price_ask'] = float(resp['asks'][0]['price'])
    data['price_spread'] = data['price_ask'] - data['price_bid']
    data['price_tick'] = (data['price_ask'] + data['price_bid']) / 2
    return()
#==========================================================================================================================


#### API Setup

In [11]:
data = {}
data_df = pd.DataFrame()

data['instrument'] = "EUR_USD"
data['pip_size'] = currs['currs'][data['instrument']]['pip_size']

access_token = config['oanda_demo_hedge']['token']
accountID = config['oanda_demo_hedge']['account_id']
params = {'instruments': data['instrument']}

api = API(access_token = access_token)
request_stream_data = pricing.PricingStream(accountID=accountID, params=params)
response_stream = api.request(request_stream_data)

## Core Engine

In [13]:
for i, resp in enumerate(response_stream): 

    if resp['type'] == 'HEARTBEAT': # Heart beat response to keep the api connection alive (Avoid timeout)
        continue

    # Date and Data ---------------------------------------------
    get_date_time(resp)

    get_prices(resp)
    print(data)

{'instrument': 'EUR_USD', 'pip_size': 0.0001, 'ts_date_val': datetime.datetime(2020, 12, 29, 0, 0), 'ts_time_val': datetime.datetime(1900, 1, 1, 4, 23, 43), 'tot_ts': '2020-Dec-29, 12:23:43 (PM)', 'time_diff': 20.563332, 'price_bid': 1.22378, 'price_ask': 1.22386, 'price_spread': 7.999999999985796e-05, 'price_tick': 1.22382}
{'instrument': 'EUR_USD', 'pip_size': 0.0001, 'ts_date_val': datetime.datetime(2020, 12, 29, 0, 0), 'ts_time_val': datetime.datetime(1900, 1, 1, 4, 23, 44), 'tot_ts': '2020-Dec-29, 12:23:44 (PM)', 'time_diff': 19.563332, 'price_bid': 1.22375, 'price_ask': 1.22383, 'price_spread': 8.000000000008001e-05, 'price_tick': 1.22379}
{'instrument': 'EUR_USD', 'pip_size': 0.0001, 'ts_date_val': datetime.datetime(2020, 12, 29, 0, 0), 'ts_time_val': datetime.datetime(1900, 1, 1, 4, 23, 46), 'tot_ts': '2020-Dec-29, 12:23:46 (PM)', 'time_diff': 17.56433, 'price_bid': 1.22376, 'price_ask': 1.22383, 'price_spread': 7.00000000000145e-05, 'price_tick': 1.223795}
{'instrument': 'EUR_

KeyboardInterrupt: 